# Competition

# Task Overview
You are given a dataset of images and descriptions for 200 categories of birds, as you could see in Assignment 2.

Download the data from [competition page](https://www.kaggle.com/t/7aeb373db52b4355bcdb3daed8e6586f). This is also where you will upload your submissions.

You need to improve the accuracy of the model as much as you can.

## Rules:
1. Do not use any external data **NOR** models pre-trained on other datasets
2. Use the test set **ONLY** to get predictions for your model. For example, do not use it to compute statistics or features (e.g. learning preprocessing).
3. Do not use deep learning models for a fair competition
4. Don't cheat :)

## Hints
Here are several techniques that you can use:

1) **Tune your hyper-parameters.** Try *GridSerachCV* function from *sklearn.model_selection* to find the best set of hyperparameters.

2) **Data pre-processing and post-processing.** In the example, we treated each caption as a different data sample. Are there any other ways to use the captions? As post-processing, we used majority voting. An alternative would be to take the most confident caption prediction (using _predict_proba_ method of the classifier, that returns probabilities)

3) **Feature engineering.** Play with the representation of the textual data. We only tried one, but there are more (e.g. TF-IDF Vectorizer is another powerful method to transform text to a vector, taking into account the rareness of the words across the texts). Also do not hesitate to play with the arguments of the *Vectorizers*. 

4) **Change your model.** You are not restricted to train *Naive Bayes* only. You can use whatever algorithm you're already familiar with. Moreover, you can use the algorithms that you get to know during these 3 weeks of solving this assignment. E.g. give *RandomForests* a try!

5) **Use image data.** The dataset consists not only of captions, but also images. You've seen examples how classifiers can be trained on images. This dataset, however, is more challenging, since the objects are not aligned. Using raw pixels might be too challenging (but not useless - give it a try!) for the classifiers we learn in class; however, there are number of methods that allow to extract image descriptors that might be better suited. Color histogram is one example. You might want to explore different types of pre-processing as well (e.g., taking image crops)

6) **Combine multiple models.** You can train multiple models (uni- or multimodal) and use their individual predictions to produce a final, improved prediction.

## Scoring rules [15 points + 20 bonus points]
You have until **14.12.2022** to submit your tuned solutions.  
**You also need to submit the code for your best solution before the deadline.**

### **Part of the Assignment grade: [15 points]**
You need to beat two thresholds in order to get a full set of points for the assignment:

- You get **5 points** if you get at least 45% on the public board

- You get another **10 points** if you beat the **hard baseline** - 55.316%. This is the score of the hidden classifier that is trained by TAs, you don't have access to its code.  
*Hint: It's a rather minor change from the simple baseline*

### **Bonus points [up to 20 points]**
- **Top-5** on the final leaderboard get **20 bonus points**

- **Top-10** on the final leaderboard get **15 bonus points**

- **Top-15** on the final leaderboard get **10 bonus points**

- **Top-25** on the final leaderboard get **5 bonus points**

In [1]:
# Try to improve! The assignment 2 code should get you started

In [2]:
import pandas as pd

# Load the data

dataset_train = pd.read_csv('train_dataframe_sorted.csv')
dataset_test = pd.read_csv('test_dataframe_obfuscated_nolabels.csv')

In [3]:
long_string = ""
for idx in range(len(dataset_test)):
    combined_descriptions = ""
    for caption_label in [f'caption_{i}' for i in range(1, 11)]:
        combined_descriptions+=" "+dataset_test[caption_label][idx]
    long_string+=combined_descriptions
    
    
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words="english")
s = vectorizer.get_stop_words()

words = long_string.replace(","," ").replace("."," ").split(" ")
words_2 = []
for i in words:
    if not i in s and not i=="":
        words_2.append(i)
words = words_2

countlst = {}
for i in words:
    if i in countlst.keys():
        countlst[i]+=1
    else:
        countlst[i]=1
        
sorted_dict = {}
top_words = sorted(countlst, key=countlst.get, reverse=True)[:30]


In [4]:
from PIL import Image
import numpy as np
# you might want to work with smaller images (but use them as you please)
def read_image(impath, size=((10,10))):
    img = np.array(Image.open(impath).resize(size))
    return np.array(img)

def convert_bw_to_col(img):
    out=[]
    for i in img:
        row = []
        for j in i:
            row.append([j,j,j])
        out.append(row)
    return np.array(out)

def check_if_bw(img):
    if np.array(img).shape == (len(img),len(img[0])):
        return True
    else:
        return False
    
def get_col_image(img):
    if check_if_bw(img):
        return convert_bw_to_col(img)
    else:
        return img
        
        

In [5]:
Image.fromarray(read_image(dataset_train['impath'][0]))

In [6]:
from sklearn.model_selection import train_test_split

dataset_train, val_split = train_test_split(dataset_train, test_size=0.1, random_state=22)
dataset_train = dataset_train.reset_index()
val_split = val_split.reset_index()


In [7]:
from scipy import stats as st
def convert_to_sample_prediction(predictions):
    '''
    Takes per-caption predictions of shape (10n,)
    Should return per-sample predictions of shape (n,)
    '''
    predictions = predictions.reshape(-1, 10) # now we have a matrix of N_samples x 10 predictions

    ### TODO: For each prediction row, return the most common one. You can use scipy.stats.mode function [2 pt]
    predictions = st.mode(predictions, axis=1).mode
    ### End of your code
    return predictions.flatten()

In [8]:
#### TODO: Data preprocessing on the train set - converting the dataset into (caption, label) pairs
#### Data preprocessing on the test set -- the test set doesn't have labels this time



train_caption_labels = {'caption': [], 'label': []}
for idx in range(len(dataset_train)):
    combined_descriptions = ""
    label_appended = False
    for caption_label in [f'caption_{i}' for i in range(1, 11)]:
        if(not label_appended):
            train_caption_labels['label'].append(dataset_train['label'][idx])
            label_appended = True
        combined_descriptions+=" "+dataset_train[caption_label][idx]
    train_caption_labels['caption'].append(combined_descriptions)
    
validation_caption_labels = {'caption': [], 'label': []}
for idx in range(len(val_split)):
    combined_descriptions = ""
    label_appended = False
    for caption_label in [f'caption_{i}' for i in range(1, 11)]:
        if(not label_appended):
            validation_caption_labels['label'].append(val_split['label'][idx])
            label_appended = True
        combined_descriptions+=" "+val_split[caption_label][idx]
    validation_caption_labels['caption'].append(combined_descriptions)
    
        
test_caption_labels = {'caption': []}
for idx in range(len(dataset_test)):
    combined_descriptions = ""
    for caption_label in [f'caption_{i}' for i in range(1, 11)]:
        combined_descriptions+=" "+dataset_test[caption_label][idx]
    test_caption_labels['caption'].append(combined_descriptions)



In [9]:
train_image_labels = {'image': [], 'label': []}
for idx in range(len(dataset_train)):
    train_image_labels['label'].append(dataset_train['label'][idx])
    image =  get_col_image(read_image(dataset_train['impath'][idx]))
    hist = image.flatten()
    #hist = np.array([np.histogram(image[:, :, i], bins=500, range=(0, 256))[0] for i in range(3)]).flatten()
    train_image_labels['image'].append(hist)
    
validation_image_labels = {'image': [], 'label': []}
for idx in range(len(val_split)):
    validation_image_labels['label'].append(val_split['label'][idx])
    image =  get_col_image(read_image(val_split['impath'][idx]))
    hist = image.flatten()
    #hist = np.array([np.histogram(image[:, :, i], bins=500, range=(0, 256))[0] for i in range(3)]).flatten()
    validation_image_labels['image'].append(hist)

        
test_image_labels = {'image': []}
for idx in range(len(dataset_test)):
    image =  get_col_image(read_image(dataset_test['impath'][idx]))
    #hist = np.array([np.histogram(image[:, :, i], bins=500, range=(0, 256))[0] for i in range(3)]).flatten()
    hist = image.flatten()
    test_image_labels['image'].append(hist)



In [10]:
train_image_labels_df = pd.DataFrame(train_image_labels)
validation_image_labels_df = pd.DataFrame(validation_image_labels)
test_image_labels_df = pd.DataFrame(test_image_labels)

In [11]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

image_train_target = train_image_labels["label"]
image_train_feautures = train_image_labels["image"]

img_clf = MultinomialNB()

img_clf.fit(image_train_feautures, image_train_target)



MultinomialNB()

In [12]:
val_set = {"caption":[], "label":[]}

import random

#### generating more training data by combining captions of different existing ones
#1. sorting the samples into pools according to their label

pools = []
for i in range(200):
    pools.append([])
    
for i in range(len(dataset_train)):
    sample = []
    for j in range(1,11):
        sample.append(dataset_train["caption_"+str(j)][i])
    pools[dataset_train["label"][i]].append(sample)
    
def generate_samples_for_class(amount, cls=0):
    #2. adding more samples for each label
    amt_of_new_sample_per_label=amount
    out = []
    i=cls
    for k in range(amt_of_new_sample_per_label):
        new_sample = ""
        for j in range(10):
            rand_sample = pools[i][random.randint(0,len(pools[i])-1)]
            rand_caption = rand_sample[random.randint(0,len(rand_sample)-1)]
            new_sample+=rand_caption
        out.append(new_sample)
    return out


In [13]:

val_set_labels_df = pd.DataFrame(val_set["label"])
val_set_captions_df = pd.DataFrame(val_set["caption"])

train_caption_labels_df = pd.DataFrame(train_caption_labels)
validation_caption_labels_df = pd.DataFrame(validation_caption_labels)
test_caption_labels_df = pd.DataFrame(test_caption_labels)

In [14]:
#### TODO: Text preprocessing - fitting the vectorizer on the train set
#Creating the vectorizer with the appropriate stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = CountVectorizer(stop_words="english")
s_words = set()
for i in vectorizer.get_stop_words():
    s_words.add(i)
irrelevant_words = ["bird"]
for i in irrelevant_words:
    s_words.add(i)  
vectorizer = CountVectorizer(ngram_range=(1,2),stop_words=s_words, min_df=11)


vectorizer.fit(train_caption_labels_df['caption'].values) # We should only use train data for pre-processing
train_caption_features = vectorizer.transform(train_caption_labels_df['caption'].values)
target_train = train_caption_labels_df['label']

validation_caption_features = vectorizer.transform(validation_caption_labels_df['caption'])
val_caption_features = vectorizer.transform(val_set["caption"])

My approach for the main classifyer is a voting classifyer, so an ensemble where the decisions is formed by aggregating the decisions of multiple MultinomialNB with different alphas. The aggregation is weighted according to the confidence of the classifyer.

(I tried many other approaches with different ensembles, boosting and different base classifyers, but this is the one that performed the best)

In [15]:
#### TODO: Train the model on the train set
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB


clf1 = MultinomialNB(alpha = 0.75)
clf2 = RandomForestClassifier(3000)
clf3 = SGDClassifier( loss="log_loss", penalty='l2')

text_classifiers = [("Multinomial Naive Bayes", clf1),
                        ("Random Forest", clf2),
                        ("SVM classifier", clf3)]

text_classifiers= []
for i in range(1,11):
    text_classifiers.append(("clf"+str(i),MultinomialNB(alpha = (0.1+0.9/i))))

text_classifiers.append(("clf12",clf1))


clf = VotingClassifier(text_classifiers, voting="soft")

clf.fit(train_caption_features, target_train)


#######################################################################
#                         END OF YOUR CODE                            #
#######################################################################

VotingClassifier(estimators=[('clf1', MultinomialNB()),
                             ('clf2', MultinomialNB(alpha=0.55)),
                             ('clf3', MultinomialNB(alpha=0.4)),
                             ('clf4', MultinomialNB(alpha=0.325)),
                             ('clf5', MultinomialNB(alpha=0.28)),
                             ('clf6', MultinomialNB(alpha=0.25)),
                             ('clf7', MultinomialNB(alpha=0.2285714285714286)),
                             ('clf8', MultinomialNB(alpha=0.21250000000000002)),
                             ('clf9', MultinomialNB(alpha=0.2)),
                             ('clf10', MultinomialNB(alpha=0.19)),
                             ('clf12', MultinomialNB(alpha=0.75))],
                 voting='soft')

#### Moddifyling with binary classifyers
The idea here stems from the fact that in 40% of the mistakes that the classifyer makes, the second most confident guess would have been the right one. So i'm trying to get a signal to decide when to use the second (or even third and so on) most confident guess. This is done with the help of 200 binary classifyers which can only distinguish between one class and all the other birds. I hope that this results in the binary classifyers being able to spot mistakes that the main classifyer makes, especially since they are two different models (the binary classifyers are passive agressive classifyers)

In [95]:
### create 200 models that make binary distinction, where the i-th classifier distingushes between the i-th bird kind and all the other kinds
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
add_more_data = True
binary_clfs = []
print("|", end="")
for i in range(48):
    print(" ", end="")
print("| 50%", end="")
for i in range(45):
    print(" ", end="")
print("| 100%")
for i in range(200):
    if i%2 ==0:
        print("|", end="")
        #print(str(i)+" binary classifiers fitted")
    labels = []
    for j in target_train:
        if j == i:
            labels.append(1)
        else:
            labels.append(0)
    if add_more_data:
        new_samples = generate_samples_for_class(100, cls = i)
        temp_training_set = [i for i in train_caption_labels["caption"]]
        for sample in new_samples:
            temp_training_set.append(sample)
            labels.append(1)
        x = np.array(temp_training_set)
        temp_feautures = vectorizer.transform(x)
        binary_clf = PassiveAggressiveClassifier(C=0.001)
        binary_clf.fit(temp_feautures, labels)
    else:
        binary_clf = PassiveAggressiveClassifier(C=0.74)
        binary_clf.fit(train_caption_features, labels)
    binary_clfs.append(binary_clf)
    

|                                                | 50%                                             | 100%
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||


from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.04)
clf.fit(train_caption_features, target_train)


In [90]:
#### TODO: Vectorize the test set and run the prediction on the test set
test_caption_features = vectorizer.transform(test_caption_labels_df['caption'].values)
predictions_test = clf.predict(test_caption_features)
predictions_validation = clf.predict(validation_caption_features)
predictions_train = clf.predict(train_caption_features)

In [91]:
#### TODO: Convert the per-caption prediction to per-sample predictions. [1 pt]
from sklearn.metrics import accuracy_score

#train_sample_predictions = convert_to_sample_prediction(predictions_train)
#test_sample_predictions = convert_to_sample_prediction(predictions_test)

train_sample_labels = train_caption_labels_df['label']
val_set_labels = val_set["label"]
validation_set_labels = validation_caption_labels["label"]
#### TODO: Report the train accuracy (should be around 55%)

train_accuracy = accuracy_score(predictions_train, train_sample_labels)
print(f"Train per-sample accuracy: {train_accuracy:.3f}")
if(len(val_set["caption"])>0):
    val_accuracy = accuracy_score(clf.predict(val_caption_features), val_set_labels)
    print(f"val per-sample accuracy: {val_accuracy:.3f}")
val_accuracy = accuracy_score(predictions_validation, validation_set_labels)
print(f"actual validation set per-sample accuracy: {val_accuracy:.3f}")



train_image_prediction = img_clf.predict(train_image_labels["image"])
test_image_prediction = img_clf.predict(test_image_labels["image"])
validation_image_prediction = img_clf.predict(validation_image_labels["image"])


img_accuracy_train = accuracy_score(train_image_prediction, train_image_labels["label"])
img_accuracy_validation = accuracy_score(validation_image_prediction, validation_image_labels["label"])
if(False):
    print("image accuracy train:", img_accuracy_train)
    print("image accuracy validation:", img_accuracy_validation)

Train per-sample accuracy: 0.977
actual validation set per-sample accuracy: 0.658


In [96]:
import matplotlib.pyplot as plt

def second_highest_index(lst):
    copy = [i for i in lst]
    m = max(copy)
    for i in range(len(copy)):
        if copy[i]==m:
            copy[i]=0
            break
    return np.argmax(copy)

def nth_highest_index(n, lst):
    copy = [(i,lst[i]) for i in range(len(lst))]
    copy.sort(key=lambda y: y[1], reverse=True)    
    return copy[n-1][0]


confidences_captions_train = clf.predict_proba(train_caption_features)
confidences_captions_validation = clf.predict_proba(validation_caption_features)
confidences_captions_test = clf.predict_proba(test_caption_features)

final_predictions = []
final_predictions_test = []
unconfirmed_decisions = 0;   

for i in range(len(confidences_captions_validation)):
    
    found = False
    pred = np.argmax(confidences_captions_validation[i])
    for j in range(2,20):
        if binary_clfs[pred].predict(validation_caption_features)[i]==0:
            pred = nth_highest_index(j,confidences_captions_validation[i])
        else:
            found = True
            #if(j>2):
                #print("found!")
            break
    
    if not found:
        pred = np.argmax(confidences_captions_validation[i])        
        unconfirmed_decisions+=1
    #if(pred != np.argmax(confidences_captions_validation[i])):
        #print("changed")
    final_predictions.append(pred)
    
print("ratio of classifications unconfirmed by the binary classifiers",unconfirmed_decisions/len(confidences_captions_validation))
    
for i in range(len(confidences_captions_test)):
    found = False
    pred = np.argmax(confidences_captions_test[i]) 
    for j in range(2,20):
        if binary_clfs[pred].predict(test_caption_features)[i]==0:
            pred = nth_highest_index(j,confidences_captions_test[i])
        else:
            found = True
            #if(j>2):
                #print("found!")
            break
            
    if not found:
        pred = np.argmax(confidences_captions_test[i])
        
    #if(pred != np.argmax(confidences_captions_test[i])):
        #print("changed")
    
    final_predictions_test.append(pred)
    
val_accuracy = accuracy_score(final_predictions, validation_set_labels)
print("validation accuracy with binary classifiers:",val_accuracy)


ratio of classifications unconfirmed by the binary classifiers 0.4181636726546906
validation accuracy with binary classifiers: 0.6656686626746507


In [97]:
if(False):
    results = []
    for i in range(1,1):
        vectorizer = CountVectorizer(ngram_range=(1,2),stop_words=s_words, min_df=i)


        vectorizer.fit(train_caption_labels_df['caption'].values) # We should only use train data for pre-processing
        train_caption_features = vectorizer.transform(train_caption_labels_df['caption'].values)
        target_train = train_caption_labels_df['label']

        validation_caption_features = vectorizer.transform(validation_caption_labels_df['caption'])
        val_caption_features = vectorizer.transform(val_set["caption"])

        clf.fit(train_caption_features, target_train)

        test_caption_features = vectorizer.transform(test_caption_labels_df['caption'].values)
        predictions_test = clf.predict(test_caption_features)
        predictions_validation = clf.predict(validation_caption_features)
        predictions_train = clf.predict(train_caption_features)

        train_sample_labels = train_caption_labels_df['label']
        val_set_labels = val_set["label"]
        validation_set_labels = validation_caption_labels["label"]
        #### TODO: Report the train accuracy (should be around 55%)

        val_accuracy = accuracy_score(predictions_validation, validation_set_labels)
        print(f"validation accuracy for min_df = {i}: {val_accuracy}")
        results.append(val_accuracy)
    print("maximales min_df",np.argmax(results)+1)
    print(max(results))
    plt.plot(results)
    plt.show()

In [98]:
#### Save the prediction in the submission format and upload to Kaggle!

########################################################################
#                                                                      #
# Create a submission DataFrame that has two columns: id and prediction#
# with the predicted label.                                            #
#                                                                      #
########################################################################

submission = pd.DataFrame(
    {"id": dataset_test['sample_id'],
     "prediction": final_predictions_test})


### Save the submission and TODO: upload to Kaggle!
submission.to_csv("simple_baseline.csv", index=None)